In [1]:
import os
import sys
import asyncio
from async_timeout import timeout

# Add the project root directory to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, project_root)

from src.clients.morpho_client import MorphoClient


async def run_with_timeout(coro, timeout_seconds=10):
    async with timeout(timeout_seconds):
        return await coro


/Users/cleme/stablecoins_optimizer/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# Setup client
RPC = "ETHEREUM"  # ARBITRUM or ETHEREUM
RPC_URL = os.getenv(f"{RPC}_RPC_URL")
user_address = os.getenv('WALLET_ADDRESS')

# Initialize client
morpho_client = MorphoClient(wallet_address=user_address,
                             private_key=os.getenv('PRIVATE_WALLET_KEY'),
                             network=RPC,
                             rpc_url=RPC_URL)

In [3]:
# Test get the smart contracts:
morpho_blue = morpho_client._get_morpho_contract()
print(morpho_blue.address)

morpho_irm = morpho_client._get_irm_contract()
print(morpho_irm.address)

0xBBBBBbbBBb9cC5e90e3b3Af64bdAF62C37EEFFCb
0x870aC11D48B15DB9a138Cf899d20F13F79Ba00BC


In [4]:
market_key = 'wstETH-USDT1' # see utils.morpho_markets for all available markets
market_id = morpho_client.get_market_info(market_key)
print(market_id)

0xe7e9694b754c4d4f7e21faf7223f6fa71abaeb10296a4c43a54a7977149687d2


In [5]:
markets_data = await run_with_timeout(morpho_client.fetch_market_data(market_id, user_address))
markets_data

market_params: MarketParams(loan_token='0xdAC17F958D2ee523a2206206994597C13D831ec7', collateral_token='0x7f39C581F595B53c5cb19bD0b3f8dA6c935E2Ca0', oracle='0x95DB30fAb9A3754e42423000DF27732CB2396992', irm='0x870aC11D48B15DB9a138Cf899d20F13F79Ba00BC', lltv=860000000000000000)
market_state: MarketState(total_supply_assets=3654739614950, total_supply_shares=3510469868396203233, total_borrow_assets=2238831283747, total_borrow_shares=2137429323384462774, last_update=1724281283, fee=0)
position_user: UserPosition(supply_shares=0, borrow_shares=0, collateral=0)
borrow_rate 880019351


{'supply_apy': 1e-18,
 'borrow_apy': 1.0000000008800194,
 'borrow_assets_user': 0,
 'market_total_supply': 5893570898700,
 'market_total_borrow': 4477662567497,
 'health_factor': 1.1579208923731619e+41,
 'is_healthy': True}